In [ ]:
from ultralytics import YOLO
import os
from IPython.display import Image, display
from IPython import display
import wandb
from wandb.integration.ultralytics import add_wandb_callback
%matplotlib inline


display.clear_output()
!yolo checks
wandb.login()

from ultralytics import settings
settings.update({"wandb": True})

In [ ]:
!curl -L https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11n.pt -o yolo11m.pt

In [ ]:
model_path = 'yolo11m.pt'
#dataset_path = '../../datasets/yolo_ipd/ipd.yaml'
dataset_path = '/media/vincent/more/bpc_teamname/datasets/yolo_ipd/ipd.yaml'

model = YOLO(model_path)

# train
model.train(data=dataset_path, epochs=1000, batch=8, project="yolo11_ipd", name="yolov11m_ipd_train_on_test")

In [ ]:
import cv2
import matplotlib.pyplot as plt
import random
import datetime
# small inference test 
inf_model_path = 'yolo11_ipd/yolov11m_ipd_train_on_test/weights/best.pt'
model = YOLO(inf_model_path)

# images
imgs = ['/media/vincent/more/bpc_teamname/datasets/ipd/train_pbr/000001/rgb_cam1/000001.jpg']
#img_dirs = ['/media/vincent/more/bpc_teamname/datasets/ipd/val/000004/rgb_cam1/']
img_dirs = ['/media/vincent/more/bpc_teamname/datasets/ipd/val/000004/rgb_cam1/', '/media/vincent/more/bpc_teamname/datasets/ipd/val/000012/rgb_cam1/']

for img_dir in img_dirs:
    for img in os.listdir(img_dir):
        imgs.append(img_dir + img)


# inference
results = model(imgs)

# display

conf_thresh = 0.0
labels = True
save_images = True


for result in results:
  for box in result.boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
    cls = int(box.cls[0])
    conf = box.conf[0]
    if conf < conf_thresh:
      continue
    label = f"{result.names[cls]}: {conf:.2f}"
    
    # Draw the box with the class derived pseudo-random color
    random.seed(cls)
    color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    cv2.rectangle(result.orig_img, (x1, y1), (x2, y2), color, 2)
    
    if labels:
      # Draw the label with white text on a colored background
      (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)
      cv2.rectangle(result.orig_img, (x1, y1 - text_height - 10), (x1 + text_width, y1), color, -1)
      cv2.putText(result.orig_img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

  # Display the image using matplotlib
  plt.figure(figsize=(10, 10))
  plt.imshow(cv2.cvtColor(result.orig_img, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.show()


  # save the images
  if save_images :
    dir = "results"
    if not os.path.exists(dir):
      os.makedirs(dir)
    
    today_and_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    orignal_name = result.path.split("/")[-1].split(".")[0]
    dest = f"{dir}/{orignal_name}_{today_and_time}.png"
    cv2.imwrite(dest, result.orig_img)
